In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-04-22


Found rundate 2024-04-21 10:06:39 < 2024-04-22 at commit 2cb45424; returning commit 06a9d1c


'2024-04-23 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-04-23 10:00:02, NJSP was reporting 190 YTD deaths
Current YTD Deaths (2025-04-22 10:00:02-04:00): 153
Previous year YTD Deaths (adjusted; 2024-04-23 10:00:02): 187.1596569366762
Projected 2025 total: 624.0587099255844
-18.3% change


0176e06c46df1341c13a86fac81525f2f7c2a403: FAUQStats cache miss: 2024, Tue Apr 23 10:00:02 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

CCODE      CNAME MCODE              MNAME HIGHWAY  \
ACCID                                                     
12918    12  Middlesex  1218    Plainsboro Twsp       1   
12923    11     Mercer  1111            Trenton     129   
12932    01   Atlantic  0112      Hamilton Twsp     446   
12933    02     Bergen  0248        Ramsey Boro     NaN   
12927    20      Union  2009        Linden City      27   
...     ...        ...   ...                ...     ...   
13136    02     Bergen  0213     Edgewater Boro     NaN   
13137    04     Camden  0436       Winslow Twsp      30   
13139    20      Union  2012    Plainfield City     NaN   
13138    04     Camden  0427    Pennsauken Twsp     130   
13140    11     Mercer  1112  Robbinsville Twsp      95   

                               LOCATION  FATALITIES  FATAL_D  FATAL_P  \
ACCID                                                                   
12918                   State Highway 1         1.0      0.0      0.0   
12923                 State Highway 129         1.0      1.0      0.0   
12932  Atlantic City Expressway MP 20.5         1.0      1.0      0.0   
12933                       East Oak St         1.0      1.0      0.0   
12927                  State Highway 27         1.0      0.0      0.0   
...                                 ...         ...      ...      ...   
13136                       Dempsey Ave         1.0      0.0      0.0   
13137                  State Highway 30         1.0      0.0      0.0   
13139                         E. 3rd St         1.0      1.0      0.0   
13138                 State Highway 130         1.0      1.0      0.0   
13140       New Jersey Turnpike MP 64.1         1.0      0.0      0.0   

       FATAL_T  FATAL_B       STREET  INJURIES                        dt  
ACCID                                                                     
12918      1.0      0.0          NaN       NaN 2024-01-01 01:56:00-05:00  
12923      0.0      0.0          NaN       NaN 2024-01-03 12:14:00-05:00  
12932      0.0      0.0          NaN       NaN 2024-01-05 09:14:00-05:00  
12933      0.0      0.0  East Oak St       NaN 2024-01-05 14:12:00-05:00  
12927      1.0      0.0          NaN       NaN 2024-01-06 08:37:00-05:00  
...        ...      ...          ...       ...                       ...  
13136      1.0      0.0  Dempsey Ave       NaN 2024-04-20 12:07:00-04:00  
13137      1.0      0.0          NaN       NaN 2024-04-20 21:46:00-04:00  
13139      0.0      0.0    E. 3rd St       NaN 2024-04-21 05:00:00-04:00  
13138      0.0      0.0          NaN       NaN 2024-04-21 18:11:00-04:00  
13140      1.0      0.0          NaN       NaN 2024-04-22 02:17:00-04:00  

[182 rows x 14 columns]

Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

driver  passenger  pedestrian  cyclist  crashes
county                                                     
Atlantic         6          0           3        0        9
Bergen           5          1           1        0        7
Burlington       7          4           3        0       13
Camden          10          0           4        0       14
Cape May         2          0           0        0        2
Cumberland       3          2           2        0        6
Essex            4          1           6        1       12
Gloucester       8          2           1        0        9
Hudson           0          0           2        0        2
Hunterdon        2          0           0        0        2
Mercer           3          0           3        0        6
Middlesex        5          1           1        0        7
Monmouth         3          1           0        0        3
Morris           3          2           0        1        3
Ocean           10          3           4        0       14
Passaic          2          2           3        1        8
Salem            2          0           2        0        4
Somerset         5          2           0        0        7
Sussex           0          1           0        0        1
Union            2          2           3        0        7
Warren           2          3           1        0        6

Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

30.5% through the year, 69.5% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

prv_ytd  prv_end  cur_ytd
county     type                              
Atlantic   driver         6       31        6
Bergen     driver         6       14        5
Burlington driver         9       32        7
Camden     driver         6       19       10
Cape May   driver         1        8        2
...                     ...      ...      ...
Salem      crashes        4       14        4
Somerset   crashes        2       18        7
Sussex     crashes        2       12        1
Union      crashes        5       35        7
Warren     crashes        4       14        6

[105 rows x 3 columns]

In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type        crashes  cyclist  driver  passenger  pedestrian
county                                                     
Atlantic         47        1      31          1          16
Bergen           27        1      13          5          10
Burlington       46        1      28          7          11
Camden           33        0      26          1           8
Cape May         12        0      11          0           1
Cumberland       27        0      18          9           9
Essex            51        2      19          7          30
Gloucester       33        0      23          8           6
Hudson           15        0       6          0          11
Hunterdon         9        0       6          1           1
Mercer           26        1      11          2          13
Middlesex        36        1      21          6           9
Monmouth         27        3      12          3          10
Morris           19        2      13          4           4
Ocean            46        1      34          8          11
Passaic          29        2      10          4          22
Salem            14        2       7          1           3
Somerset         35        0      27          3           5
Sussex            9        1       4          3           1
Union            41        1      17          8          14
Warren           18        0      11          6           2

In [9]:
projected.to_csv(PROJECTED_CSV)